<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/07_pytorch_experiment_tracking_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07. PyTorch Experiment Tracking Exercise Template

Welcome to the 07. PyTorch Experiment Tracking exercise template notebook.

> **Note:** There may be more than one solution to each of the exercises. This notebook only shows one possible example.

## Resources

1. These exercises/solutions are based on [section 07. PyTorch Transfer Learning](https://www.learnpytorch.io/07_pytorch_experiment_tracking/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.
2. See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/cO_r2FYcAjU).
3. See [other solutions on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions).

> **Note:** The first section of this notebook is dedicated to getting various helper functions and datasets used for the exercises. The exercises start at the heading "Exercise 1: ...".

### Get various imports and helper functions

We'll need to make sure we have `torch` v.1.12+ and `torchvision` v0.13+.

In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U --pre torch torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.0 MB/s eta 0:00:00
  ERROR: HTTP error 403 while getting https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from https://download.pytorch.org/whl/nightly/cu113/nvidia-cuda-nvrtc-cu12/)
ERROR: Could not install requirement nvidia-cuda-nvrtc-cu12==12.1.105 from https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from torch) because of HTTP error 403 Client Error: Forbidden for url: https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl for URL https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-ma

In [2]:
 # Make sure we have a GPU
 device = "cuda" if torch.cuda.is_available() else "cpu"
 device

'cuda'

In [3]:
# Get regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4056, done.
remote: Total 4056 (delta 0), reused 0 (delta 0), pack-reused 4056
Receiving objects: 100% (4056/4056), 646.90 MiB | 38.87 MiB/s, done.
Resolving deltas: 100% (2371/2371), done.
Updating files: 100% (248/248), done.


In [4]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [5]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from source and unzips to destination.

    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.

    Returns:
        pathlib.Path to downloaded data.

    Example usage:
        download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                      destination="pizza_steak_sushi")
    """
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it...
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...")
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)

    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] Did not find data/pizza_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


PosixPath('data/pizza_steak_sushi')

In [6]:
from torch.utils.tensorboard import SummaryWriter
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None):
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [7]:
# Create a test writer
writer = create_writer(experiment_name="test_experiment_name",
                       model_name="this_is_the_model_name",
                       extra="add_a_little_extra_if_you_want")

[INFO] Created SummaryWriter, saving to: runs/2024-06-17/test_experiment_name/this_is_the_model_name/add_a_little_extra_if_you_want...


In [8]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step

# Add writer parameter to train()
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter # new parameter to take in a writer
          ) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Stores metrics to specified writer log_dir if present.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      writer: A SummaryWriter() instance to log model results to.

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]}
      For example if training for epochs=2:
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc},
                               global_step=epoch)

            # Close the writer
            writer.close()
        else:
            pass
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

### Download data

Using the same data from https://www.learnpytorch.io/07_pytorch_experiment_tracking/

In [9]:
# Download 10 percent and 20 percent training data (if necessary)
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

[INFO] data/pizza_steak_sushi directory exists, skipping download.
[INFO] Did not find data/pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip data...


In [10]:
# Setup training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup testing directory paths (note: use the same test dataset for both to compare the results)
test_dir = data_10_percent_path / "test"

# Check the directories
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")

Training directory 10%: data/pizza_steak_sushi/train
Training directory 20%: data/pizza_steak_sushi_20_percent/train
Testing directory: data/pizza_steak_sushi/test


In [11]:
from torchvision import transforms

# Create a transform to normalize data distribution to be inline with ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], # values per colour channel [red, green, blue]
                                 std=[0.229, 0.224, 0.225])

# Create a transform pipeline
simple_transform = transforms.Compose([
                                       transforms.Resize((224, 224)),
                                       transforms.ToTensor(), # get image values between 0 & 1
                                       normalize
])

### Turn data into DataLoaders

In [12]:
BATCH_SIZE = 32

# Create 10% training and test DataLoaders
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE)

# Create 20% training and test DataLoaders
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE)

# Find the number of samples/batches per dataloader (using the same test_dataloader for both experiments)
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(train_dataloader_10_percent)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 8 (all experiments will use the same test set)
Number of classes: 3, class names: ['pizza', 'steak', 'sushi']


## Exercise 1: Pick a larger model from [`torchvision.models`](https://pytorch.org/vision/main/models.html) to add to the list of experiments (for example, EffNetB3 or higher)

* How does it perform compared to our existing models?
* **Hint:** You'll need to set up an exerpiment similar to [07. PyTorch Experiment Tracking section 7.6](https://www.learnpytorch.io/07_pytorch_experiment_tracking/#76-create-experiments-and-set-up-training-code).

In [13]:
import torchvision.models as models
weight = models.EfficientNet_B4_Weights.DEFAULT # .DEFAULT = "best available weights"
effnet4 = models.efficientnet_b4(weights=weight)

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 87.7MB/s]


In [22]:
import torch
import torchvision
from torch import nn
from pathlib import Path
from torchinfo import summary
from going_modular.going_modular.utils import save_model
from timeit import default_timer as timer
from torch.utils.tensorboard import SummaryWriter

# Define utility functions
def set_seeds(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def create_model(model_name, out_features=len(class_names)):
    if model_name == "effnetb2":
        weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
        model = torchvision.models.efficientnet_b2(weights=weights).to(device)
        dropout = 0.3
        in_features = 1408
    elif model_name == "effnetb4":
        weights = torchvision.models.EfficientNet_B4_Weights.DEFAULT
        model = torchvision.models.efficientnet_b4(weights=weights).to(device)
        dropout = 0.2
        in_features = 1792

    # Freeze base model layers
    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    # Update classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=dropout, inplace=True),
        nn.Linear(in_features=in_features, out_features=out_features)
    ).to(device)

    model.name = model_name
    print(f"[INFO] Creating {model.name} feature extractor model...")
    return model

def create_writer(experiment_name, model_name, extra=""):
    log_dir = Path("runs") / experiment_name / model_name / extra
    return SummaryWriter(log_dir=str(log_dir))

# Set the random seeds
set_seeds(seed=42)

# Define experiment parameters
num_epochs = [5, 10]
models = ["effnetb2", "effnetb4"]
train_dataloaders = {
    "data_10_percent": train_dataloader_10_percent,
    "data_20_percent": train_dataloader_20_percent
}

# Start experiment
experiment_number = 0
for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in models:
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")

            model = create_model(model_name)

            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

            # Start the timer
            start_time = timer()

            # Train the model
            train(model=model,train_dataloader=train_dataloader,test_dataloader=test_dataloader,optimizer=optimizer,loss_fn=loss_fn,
                  epochs=epochs,device=device,writer=create_writer(experiment_name=dataloader_name, model_name=model_name, extra=f"{epochs}_epochs"))

            # End the timer
            end_time = timer()
            print(f"[INFO] Total training time for {model_name} with {dataloader_name}: {end_time-start_time:.3f} seconds")

            # Save the model
            save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model, target_dir="models", model_name=save_filepath)
            print("###########" * 100 + "\n")


[INFO] Experiment number: 1
[INFO] Model: effnetb2
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Creating effnetb2 feature extractor model...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0845 | train_acc: 0.3984 | test_loss: 0.9591 | test_acc: 0.6903
Epoch: 2 | train_loss: 0.9241 | train_acc: 0.6406 | test_loss: 0.8659 | test_acc: 0.8144
Epoch: 3 | train_loss: 0.8330 | train_acc: 0.7344 | test_loss: 0.7461 | test_acc: 0.9384
Epoch: 4 | train_loss: 0.7067 | train_acc: 0.8906 | test_loss: 0.7138 | test_acc: 0.8778
Epoch: 5 | train_loss: 0.6750 | train_acc: 0.7891 | test_loss: 0.7052 | test_acc: 0.8873
[INFO] Total training time for effnetb2 with data_10_percent: 10.634 seconds
[INFO] Saving model to: models/07_effnetb2_data_10_percent_5_epochs.pth
######################################################################################################################################################################################################################################################################################################################################################################################################################

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0788 | train_acc: 0.4141 | test_loss: 1.0573 | test_acc: 0.5388
Epoch: 2 | train_loss: 1.0290 | train_acc: 0.6719 | test_loss: 1.0069 | test_acc: 0.5284
Epoch: 3 | train_loss: 0.9087 | train_acc: 0.6992 | test_loss: 0.9833 | test_acc: 0.6203
Epoch: 4 | train_loss: 1.1181 | train_acc: 0.6523 | test_loss: 0.9706 | test_acc: 0.5085
Epoch: 5 | train_loss: 0.9127 | train_acc: 0.7344 | test_loss: 0.8467 | test_acc: 0.8958
[INFO] Total training time for effnetb4 with data_10_percent: 11.852 seconds
[INFO] Saving model to: models/07_effnetb4_data_10_percent_5_epochs.pth
######################################################################################################################################################################################################################################################################################################################################################################################################################

...

Epoch: 1 | train_loss: 1.0237 | train_acc: 0.6000 | test_loss: 0.9353 | test_acc: 0.8352
Epoch: 2 | train_loss: 0.8431 | train_acc: 0.8604 | test_loss: 0.7910 | test_acc: 0.8655
Epoch: 3 | train_loss: 0.7396 | train_acc: 0.8833 | test_loss: 0.6962 | test_acc: 0.9167
Epoch: 4 | train_loss: 0.6479 | train_acc: 0.8812 | test_loss: 0.6319 | test_acc: 0.9167
Epoch: 5 | train_loss: 0.6014 | train_acc: 0.8750 | test_loss: 0.5875 | test_acc: 0.9167
[INFO] Total training time for effnetb4 with data_20_percent: 18.202 seconds
[INFO] Saving model to: models/07_effnetb4_data_20_percent_5_epochs.pth
######################################################################################################################################################################################################################################################################################################################################################################################################################

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9896 | train_acc: 0.5208 | test_loss: 0.7830 | test_acc: 0.8049
Epoch: 2 | train_loss: 0.7384 | train_acc: 0.7646 | test_loss: 0.6643 | test_acc: 0.8769
Epoch: 3 | train_loss: 0.6080 | train_acc: 0.8187 | test_loss: 0.5624 | test_acc: 0.9384
Epoch: 4 | train_loss: 0.5470 | train_acc: 0.8479 | test_loss: 0.5689 | test_acc: 0.8570
Epoch: 5 | train_loss: 0.4403 | train_acc: 0.8708 | test_loss: 0.4497 | test_acc: 0.9489
Epoch: 6 | train_loss: 0.3917 | train_acc: 0.9062 | test_loss: 0.4616 | test_acc: 0.8873
Epoch: 7 | train_loss: 0.3688 | train_acc: 0.9104 | test_loss: 0.4249 | test_acc: 0.9384
Epoch: 8 | train_loss: 0.3850 | train_acc: 0.8938 | test_loss: 0.4438 | test_acc: 0.8674
Epoch: 9 | train_loss: 0.3273 | train_acc: 0.9250 | test_loss: 0.4338 | test_acc: 0.8977
Epoch: 10 | train_loss: 0.3733 | train_acc: 0.8958 | test_loss: 0.3969 | test_acc: 0.9280
[INFO] Total training time for effnetb2 with data_20_percent: 31.579 seconds
[INFO] Saving model to: models/0

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0237 | train_acc: 0.6000 | test_loss: 0.9353 | test_acc: 0.8352
Epoch: 2 | train_loss: 0.8431 | train_acc: 0.8604 | test_loss: 0.7910 | test_acc: 0.8655
Epoch: 3 | train_loss: 0.7396 | train_acc: 0.8833 | test_loss: 0.6962 | test_acc: 0.9167
Epoch: 4 | train_loss: 0.6479 | train_acc: 0.8812 | test_loss: 0.6319 | test_acc: 0.9167
Epoch: 5 | train_loss: 0.6014 | train_acc: 0.8750 | test_loss: 0.5875 | test_acc: 0.9167
Epoch: 6 | train_loss: 0.5700 | train_acc: 0.8729 | test_loss: 0.5714 | test_acc: 0.9271
Epoch: 7 | train_loss: 0.5326 | train_acc: 0.8750 | test_loss: 0.5838 | test_acc: 0.8561
Epoch: 8 | train_loss: 0.4876 | train_acc: 0.8917 | test_loss: 0.5348 | test_acc: 0.8968
Epoch: 9 | train_loss: 0.4497 | train_acc: 0.9167 | test_loss: 0.4924 | test_acc: 0.8968
Epoch: 10 | train_loss: 0.4136 | train_acc: 0.9167 | test_loss: 0.4894 | test_acc: 0.8456
[INFO] Total training time for effnetb4 with data_20_percent: 37.666 seconds
[INFO] Saving model to: models/0

## Exercise 2. Introduce data augmentation to the list of experiments using the 20% pizza, steak, sushi training and test datasets, does this change anything?
    
* For example, you could have one training DataLoader that uses data augmentation (e.g. `train_dataloader_20_percent_aug` and `train_dataloader_20_percent_no_aug`) and then compare the results of two of the same model types training on these two DataLoaders.
* **Note:** You may need to alter the `create_dataloaders()` function to be able to take a transform for the training data and the testing data (because you don't need to perform data augmentation on the test data). See [04. PyTorch Custom Datasets section 6](https://www.learnpytorch.io/04_pytorch_custom_datasets/#6-other-forms-of-transforms-data-augmentation) for examples of using data augmentation or the script below for an example:

```python
# Note: Data augmentation transform like this should only be performed on training data
train_transform_data_aug = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    normalize
])

# Create a helper function to visualize different augmented (and not augmented) images
def view_dataloader_images(dataloader, n=10):
    if n > 10:
        print(f"Having n higher than 10 will create messy plots, lowering to 10.")
        n = 10
    imgs, labels = next(iter(dataloader))
    plt.figure(figsize=(16, 8))
    for i in range(n):
        # Min max scale the image for display purposes
        targ_image = imgs[i]
        sample_min, sample_max = targ_image.min(), targ_image.max()
        sample_scaled = (targ_image - sample_min)/(sample_max - sample_min)

        # Plot images with appropriate axes information
        plt.subplot(1, 10, i+1)
        plt.imshow(sample_scaled.permute(1, 2, 0)) # resize for Matplotlib requirements
        plt.title(class_names[labels[i]])
        plt.axis(False)

# Have to update `create_dataloaders()` to handle different augmentations
import os
from torch.utils.data import DataLoader
from torchvision import datasets

NUM_WORKERS = os.cpu_count() # use maximum number of CPUs for workers to load data

# Note: this is an update version of data_setup.create_dataloaders to handle
# differnt train and test transforms.
def create_dataloaders(
    train_dir,
    test_dir,
    train_transform, # add parameter for train transform (transforms on train dataset)
    test_transform,  # add parameter for test transform (transforms on test dataset)
    batch_size=32, num_workers=NUM_WORKERS
):
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names
```

In [27]:
import torch
import torchvision
from torch import nn
from pathlib import Path
from torchinfo import summary
from going_modular.going_modular.utils import save_model
from timeit import default_timer as timer
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_aug_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    normalize
])
no_data_aug_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir, test_dir, train_transform, test_transform, batch_size, num_workers=NUM_WORKERS):
    """Creates and returns training and testing DataLoaders."""
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)

    class_names = train_data.classes

    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_dataloader, test_dataloader, class_names

# Experiment setup
train_20_percent_dir = image_path / "train"
test_20_percent_dir = image_path / "test"
BATCH_SIZE = 32

train_dataloader_20_percent_with_aug, test_dataloader_20_percent, class_names = create_dataloaders(
    train_dir=train_20_percent_dir,
    test_dir=test_20_percent_dir,
    train_transform=data_aug_transform,
    test_transform=no_data_aug_transform,
    batch_size=BATCH_SIZE
)

train_dataloader_20_percent_without_aug, test_dataloader_20_percent, class_names = create_dataloaders(
    train_dir=train_20_percent_dir,
    test_dir=test_20_percent_dir,
    train_transform=no_data_aug_transform,
    test_transform=no_data_aug_transform,
    batch_size=BATCH_SIZE
)


num_epochs = [5, 10]
models = ["effnetb2", "effnetb4"]
train_dataloaders = {
    "train_dataloader_20_percent_with_aug": train_dataloader_20_percent_with_aug,
    "train_dataloader_20_percent_without_aug": train_dataloader_20_percent_without_aug
}

set_seeds(seed=42)

experiment_number = 0
for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in models:
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")

            model = create_model(model_name)

            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

            start_time = timer()
            train(
                model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=epochs,
                device=device,
                writer=create_writer(experiment_name=dataloader_name, model_name=model_name, extra=f"{epochs}_epochs")
            )
            end_time = timer()
            print(f"[INFO] Total training time for {model_name} with {dataloader_name}: {end_time - start_time:.3f} seconds")

            save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model, target_dir="models", model_name=save_filepath)
            print("###########" * 100 + "\n")

[INFO] Experiment number: 1
[INFO] Model: effnetb2
[INFO] DataLoader: train_dataloader_20_percent_with_aug
[INFO] Number of epochs: 5
[INFO] Creating effnetb2 feature extractor model...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0926 | train_acc: 0.3438 | test_loss: 0.9468 | test_acc: 0.7519
Epoch: 2 | train_loss: 0.9524 | train_acc: 0.5547 | test_loss: 0.8875 | test_acc: 0.7424
Epoch: 3 | train_loss: 0.8795 | train_acc: 0.6367 | test_loss: 0.7675 | test_acc: 0.8561
Epoch: 4 | train_loss: 0.7571 | train_acc: 0.8359 | test_loss: 0.7291 | test_acc: 0.8977
Epoch: 5 | train_loss: 0.7358 | train_acc: 0.7305 | test_loss: 0.7162 | test_acc: 0.8873
[INFO] Total training time for effnetb2 with train_dataloader_20_percent_with_aug: 11.467 seconds
[INFO] Saving model to: models/07_effnetb2_train_dataloader_20_percent_with_aug_5_epochs.pth
############################################################################################################################################################################################################################################################################################################################################################################

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0779 | train_acc: 0.3867 | test_loss: 1.0597 | test_acc: 0.5597
Epoch: 2 | train_loss: 1.0277 | train_acc: 0.6484 | test_loss: 1.0047 | test_acc: 0.5795
Epoch: 3 | train_loss: 0.9305 | train_acc: 0.7109 | test_loss: 0.9604 | test_acc: 0.6515
Epoch: 4 | train_loss: 1.1276 | train_acc: 0.6367 | test_loss: 0.9445 | test_acc: 0.5492
Epoch: 5 | train_loss: 0.9460 | train_acc: 0.6953 | test_loss: 0.8111 | test_acc: 0.8759
[INFO] Total training time for effnetb4 with train_dataloader_20_percent_with_aug: 12.948 seconds
[INFO] Saving model to: models/07_effnetb4_train_dataloader_20_percent_with_aug_5_epochs.pth
############################################################################################################################################################################################################################################################################################################################################################################

...

Epoch: 1 | train_loss: 1.0788 | train_acc: 0.4141 | test_loss: 1.0573 | test_acc: 0.5388
Epoch: 2 | train_loss: 1.0290 | train_acc: 0.6719 | test_loss: 1.0069 | test_acc: 0.5284
Epoch: 3 | train_loss: 0.9087 | train_acc: 0.6992 | test_loss: 0.9833 | test_acc: 0.6203
Epoch: 4 | train_loss: 1.1181 | train_acc: 0.6523 | test_loss: 0.9706 | test_acc: 0.5085
Epoch: 5 | train_loss: 0.9127 | train_acc: 0.7344 | test_loss: 0.8467 | test_acc: 0.8958
[INFO] Total training time for effnetb4 with train_dataloader_20_percent_without_aug: 11.944 seconds
[INFO] Saving model to: models/07_effnetb4_train_dataloader_20_percent_without_aug_5_epochs.pth
######################################################################################################################################################################################################################################################################################################################################################################

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0845 | train_acc: 0.3984 | test_loss: 0.9591 | test_acc: 0.6903
Epoch: 2 | train_loss: 0.9241 | train_acc: 0.6406 | test_loss: 0.8659 | test_acc: 0.8144
Epoch: 3 | train_loss: 0.8330 | train_acc: 0.7344 | test_loss: 0.7461 | test_acc: 0.9384
Epoch: 4 | train_loss: 0.7067 | train_acc: 0.8906 | test_loss: 0.7138 | test_acc: 0.8778
Epoch: 5 | train_loss: 0.6750 | train_acc: 0.7891 | test_loss: 0.7052 | test_acc: 0.8873
Epoch: 6 | train_loss: 0.6031 | train_acc: 0.7773 | test_loss: 0.6310 | test_acc: 0.9280
Epoch: 7 | train_loss: 0.5891 | train_acc: 0.8008 | test_loss: 0.6354 | test_acc: 0.8873
Epoch: 8 | train_loss: 0.5280 | train_acc: 0.9336 | test_loss: 0.6177 | test_acc: 0.8873
Epoch: 9 | train_loss: 0.5382 | train_acc: 0.8125 | test_loss: 0.6268 | test_acc: 0.8466
Epoch: 10 | train_loss: 0.4938 | train_acc: 0.9180 | test_loss: 0.5865 | test_acc: 0.8873
[INFO] Total training time for effnetb2 with train_dataloader_20_percent_without_aug: 21.363 seconds
[INFO] S

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0788 | train_acc: 0.4141 | test_loss: 1.0573 | test_acc: 0.5388
Epoch: 2 | train_loss: 1.0290 | train_acc: 0.6719 | test_loss: 1.0069 | test_acc: 0.5284
Epoch: 3 | train_loss: 0.9087 | train_acc: 0.6992 | test_loss: 0.9833 | test_acc: 0.6203
Epoch: 4 | train_loss: 1.1181 | train_acc: 0.6523 | test_loss: 0.9706 | test_acc: 0.5085
Epoch: 5 | train_loss: 0.9127 | train_acc: 0.7344 | test_loss: 0.8467 | test_acc: 0.8958
Epoch: 6 | train_loss: 0.8248 | train_acc: 0.7773 | test_loss: 0.7993 | test_acc: 0.9167
Epoch: 7 | train_loss: 0.8121 | train_acc: 0.7656 | test_loss: 0.7988 | test_acc: 0.8362
Epoch: 8 | train_loss: 0.8421 | train_acc: 0.7773 | test_loss: 0.7825 | test_acc: 0.8049
Epoch: 9 | train_loss: 0.7917 | train_acc: 0.7812 | test_loss: 0.7639 | test_acc: 0.8258
Epoch: 10 | train_loss: 0.7338 | train_acc: 0.7930 | test_loss: 0.7791 | test_acc: 0.8655
[INFO] Total training time for effnetb4 with train_dataloader_20_percent_without_aug: 23.534 seconds
[INFO] S

## Exercise 3. Scale up the dataset to turn FoodVision Mini into FoodVision Big using the entire [Food101 dataset from `torchvision.models`](https://pytorch.org/vision/stable/generated/torchvision.datasets.Food101.html#torchvision.datasets.Food101)
    
* You could take the best performing model from your various experiments or even the EffNetB2 feature extractor we created in this notebook and see how it goes fitting for 5 epochs on all of Food101.
* If you try more than one model, it would be good to have the model's results tracked.
* If you load the Food101 dataset from `torchvision.models`, you'll have to create PyTorch DataLoaders to use it in training.
* **Note:** Due to the larger amount of data in Food101 compared to our pizza, steak, sushi dataset, this model will take longer to train.

In [ ]:
train_dataloader_20_percent_with_aug, test_dataloader_20_percent, class_names = create_dataloaders(
    train_dir=train_20_percent_dir,
    test_dir=test_20_percent_dir,
    train_transform=data_aug_transform,
    test_transform=no_data_aug_transform,
    batch_size=BATCH_SIZE
)

train_dataloader_20_percent_without_aug, test_dataloader_20_percent, class_names = create_dataloaders(
    train_dir=train_20_percent_dir,
    test_dir=test_20_percent_dir,
    train_transform=no_data_aug_transform,
    test_transform=no_data_aug_transform,
    batch_size=BATCH_SIZE
)


num_epochs = [10]
models = ["effnetb2", "effnetb4"]
train_dataloaders = {
    "train_dataloader_20_percent_with_aug": train_dataloader_20_percent_with_aug,
    "train_dataloader_20_percent_without_aug": train_dataloader_20_percent_without_aug
}

set_seeds(seed=42)

experiment_number = 0
for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in models:
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")

            model = create_model(model_name)

            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

            start_time = timer()
            train(
                model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=epochs,
                device=device,
                writer=create_writer(experiment_name=dataloader_name, model_name=model_name, extra=f"{epochs}_epochs")
            )
            end_time = timer()
            print(f"[INFO] Total training time for {model_name} with {dataloader_name}: {end_time - start_time:.3f} seconds")

            save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model, target_dir="models", model_name=save_filepath)
            print("###########" * 100 + "\n")

In [28]:
# Get Food101 Dataset
import torchvision
# In last exerc we have transform no_data_aug_transform
train_data = torchvision.datasets.Food101(root="data",
                                          split="train",
                                          transform=no_data_aug_transform,
                                          download=True)

test_data = torchvision.datasets.Food101(root="data",
                                        split="test",
                                        transform=no_data_aug_transform,
                                        download=True)

100%|██████████| 4996278331/4996278331 [03:55<00:00, 21210930.54it/s]


Extracting data/food-101.tar.gz to data


In [29]:
import os
BATCH_SIZE = 512

new_train_dataloader = torch.utils.data.DataLoader(train_data,
                                                   shuffle=True,
                                                   batch_size=BATCH_SIZE,
                                                   num_workers=os.cpu_count(),
                                                   pin_memory=True)

new_test_dataloader = torch.utils.data.DataLoader(test_data,
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE,
                                                  num_workers=os.cpu_count(),
                                                  pin_memory=True)

In [33]:
weight = models.EfficientNet_B4_Weights.DEFAULT
effnet4 = models.efficientnet_b4(weights=weight).to(device)
# Freeze the base layers
for param in effnet4.features.parameters():
  param.requires_grad = False

# Change the classifier head to suit 101 different classes
effnet4.classifier = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(in_features=1792, out_features=101)
).to(device)

In [ ]:
results = train(model=effnet4,train_dataloader=new_train_dataloader,test_dataloader=new_test_dataloader,optimizer=torch.optim.Adam(params=effnet4.parameters(), lr=0.001),
                loss_fn=torch.nn.CrossEntropyLoss(),epochs=5,device=device,writer=create_writer(experiment_name="all_data",model_name="Result_fuLL_model",extra=f"{epochs}_epochs"))

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.8724 | train_acc: 0.2977 | test_loss: 3.1763 | test_acc: 0.4238
Epoch: 2 | train_loss: 3.0067 | train_acc: 0.3940 | test_loss: 2.6012 | test_acc: 0.4664
Epoch: 3 | train_loss: 2.6351 | train_acc: 0.4286 | test_loss: 2.3346 | test_acc: 0.4887
